In [2]:
import matplotlib.pyplot as plt
import pandas as pd

With 100 points, three cases:

1. Ideal
2. Gaussian noise
3. Uniform noise

we have created a dataset with 100 points, with 5 features each. The relation we want to investigate is

$$
2.5382 cos(x_3)+x_0^2-0.5
$$

In [3]:
import numpy as np

X = 2 * np.random.randn(100, 5)
a = 2 #weight for noise
y = 2.5382 * np.cos(X[:,3]) + X[:,0] ** 2 - 0.5 
n = y + a*np.random.randn(100,1)
u = y + a*np.random.rand(100,1)

In [4]:
from pysr import PySRRegressor

model = PySRRegressor(
    niterations=40,  # < Increase me for better results
    binary_operators=["+", "*"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "inv(x) = 1/x",
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)

## 1. Ideal


In [5]:
model.fit(X, y)

/Users/francescofarinelli/miniconda3/envs/simreg/lib/python3.11/site-packages/pysr/sr.py:1296: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Compiling Julia backend...


/Users/francescofarinelli/miniconda3/envs/simreg/lib/python3.11/site-packages/pysr/julia_helpers.py:208: UserWarning: Your system's Python library is static (e.g., conda), so precompilation will be turned off. For a dynamic library, try using `pyenv` and installing with `--enable-shared`: https://github.com/pyenv/pyenv/blob/master/plugins/python-build/README.md#building-with---enable-shared.
  warnings.warn(


Started!

Expressions evaluated per second: 1.800e+05
Head worker occupation: 15.1%
Progress: 399 / 600 total iterations (66.500%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           3.959e+01  1.926e-07  3.7029111
3           3.298e+00  1.243e+00  (x0 * x0)
5           3.258e+00  6.170e-03  ((x0 * 0.9723541) * x0)
6           1.252e+00  9.568e-01  ((x0 * x0) + cos(x3))
8           2.327e-01  8.411e-01  ((x0 * x0) + (cos(x3) * 2.3610847))
10          1.777e-13  1.151e+01  (((x0 * x0) + (cos(x3) * 2.5382)) + -0.50000006)
18          8.012e-14  9.959e-02  (((((x0 * x0) + (cos(x3) * ((1.6912384 + 0.4028847) + 0.444076...
                                  9))) + -0.47816002) + -0.1821089) + 0.16026881)
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.


PySRRegressor.equations_ = [
	   pick      score                                           equation  \
	0         0.000000                                          3.7029111   
	1         1.242516                                          (x0 * x0)   
	2         0.006170                            ((x0 * 0.9723541) * x0)   
	3         0.956754                              ((x0 * x0) + cos(x3))   
	4         0.841077                ((x0 * x0) + (cos(x3) * 2.3610847))   
	5        13.950335   (((x0 * x0) + (cos(x3) * 2.5382)) + -0.50000006)   
	6  >>>>   0.099594  (((((x0 * x0) + (cos(x3) * ((1.6912384 + 0.402...   
	
	           loss  complexity  
	0  3.958632e+01           1  
	1  3.298445e+00           3  
	2  3.257994e+00           5  
	3  1.251518e+00           6  
	4  2.327486e-01           8  
	5  1.777378e-13          10  
	6  8.012257e-14          18  
]

In [10]:
df = pd.read_csv('hall_of_fame_2023-08-13_184043.953.csv')
# barplot coplexity vs score (da capì)
plt.bar(df['Complexity'],df[1/df['Loss']])
plt.logscale()

KeyError: "None of [Float64Index([0.025261251866806514,  0.30317314351622393,   0.3069373645504594,\n                0.7990296583828599,    4.296481821563931,    5626264256742.642,\n                12480877112110.752],\n             dtype='float64')] are in the [columns]"

### Detailed Example

The following code makes use of as many PySR features as possible. Note that is just a demonstration of features and you should not use this example as-is. For details on what each parameter does, check out the API page.

In [ ]:
model = PySRRegressor(
    procs=4,
    populations=8,
    # ^ 2 populations per core, so one is always running.
    population_size=50,
    # ^ Slightly larger populations, for greater diversity.
    ncyclesperiteration=500, 
    # ^ Generations between migrations.
    niterations=10000000,  # Run forever
    early_stop_condition=(
        "stop_if(loss, complexity) = loss < 1e-6 && complexity < 10"
        # Stop early if we find a good and simple equation
    ),
    timeout_in_seconds=60 * 60 * 24,
    # ^ Alternatively, stop after 24 hours have passed.
    maxsize=50,
    # ^ Allow greater complexity.
    maxdepth=10,
    # ^ But, avoid deep nesting.
    binary_operators=["*", "+", "-", "/"],
    unary_operators=["square", "cube", "exp", "cos2(x)=cos(x)^2"],
    constraints={
        "/": (-1, 9),
        "square": 9,
        "cube": 9,
        "exp": 9,
    },
    # ^ Limit the complexity within each argument.
    # "inv": (-1, 9) states that the numerator has no constraint,
    # but the denominator has a max complexity of 9.
    # "exp": 9 simply states that `exp` can only have
    # an expression of complexity 9 as input.
    nested_constraints={
        "square": {"square": 1, "cube": 1, "exp": 0},
        "cube": {"square": 1, "cube": 1, "exp": 0},
        "exp": {"square": 1, "cube": 1, "exp": 0},
    },
    # ^ Nesting constraints on operators. For example,
    # "square(exp(x))" is not allowed, since "square": {"exp": 0}.
    complexity_of_operators={"/": 2, "exp": 3},
    # ^ Custom complexity of particular operators.
    complexity_of_constants=2,
    # ^ Punish constants more than variables
    select_k_features=4,
    # ^ Train on only the 4 most important features
    progress=True,
    # ^ Can set to false if printing to a file.
    weight_randomize=0.1,
    # ^ Randomize the tree much more frequently
    cluster_manager=None,
    # ^ Can be set to, e.g., "slurm", to run a slurm
    # cluster. Just launch one script from the head node.
    precision=64,
    # ^ Higher precision calculations.
    warm_start=True,
    # ^ Start from where left off.
    turbo=True,
    # ^ Faster evaluation (experimental)
    julia_project=None,
    # ^ Can set to the path of a folder containing the
    # "SymbolicRegression.jl" repo, for custom modifications.
    update=False,
    # ^ Don't update Julia packages
    extra_sympy_mappings={"cos2": lambda x: sympy.cos(x)**2},
    # extra_torch_mappings={sympy.cos: torch.cos},
    # ^ Not needed as cos already defined, but this
    # is how you define custom torch operators.
    # extra_jax_mappings={sympy.cos: "jnp.cos"},
    # ^ For JAX, one passes a string.
)